In [46]:
## Libraries
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import yfinance as yf
import datetime
from matplotlib import pyplot
from googlesearch import search

## Basic Settings
pd.options.display.max_colwidth = 90
pd.set_option('display.max_rows', 500)

In [48]:
# Empresas para Analisar

# empresas = ["MGLU3","KLBN11","BRKM5","SUZB3","PETR4","VALE3","LOGN3","VLID3","ABEV3","RLOG3","POMO4","GOAU4","SCAR3",'ELET3','IRBR3','MOVI3']

# setorAviacao = ["AZUL4", "SMLS3","GOLL4","CVCB3"]
setorCarne = ['BRFS3','JBSS3','MRFG3','BEEF3','MNPR3']
# setorSaneamento = ['AMBP3','SBSP3','CSMG3','IGSN3']
# setorBanco = ['ITUB4', 'BBDC4']
# setorCarros = ['RENT3', 'B3SA3','PCAR4','LREN3','NATU3','ITUB4', 'BBDC4']

empresas = setorCarne

In [50]:
# Creating Dictionary for the DataFrames
frames = {}

links = []

# to search
for i in empresas:
    query = "advfn "+ i +" fundamentos"
    l = str(search(query, 0))
    
    for k in ['[',']',"'"]:
        l = l.replace(k,"")
    links.append(l)


for urlLink,compName in zip(links, empresas):
    
    # Company Name
    dataframeName = compName
    
    # creating the lists
    labels = []
    data = []
    i = 0
    
    # loop for informations
    result = requests.get(urlLink)
    c = result.content
    soup = BeautifulSoup(c,'lxml')
    
    for tr in soup.find_all('tr'):
        i += 1
        if(i < 75):
            # finding labels
            lab = tr.findChildren()[0].string
            labels.append(lab)

            #finding data
            dat = tr.findChildren()[1].string
            data.append(dat.replace(",","."))

    
    # Preparing DataFrame
    tupleDados = list(zip(labels,data))
    frameDados = pd.DataFrame(tupleDados, columns=['Label','Valor'])
    frameDados['Valor'] = frameDados['Valor'].str.replace('\n','')

    # Final DataFrame
    frames[dataframeName] = frameDados
    print('sucesso '+ dataframeName)

sucesso BRFS3
sucesso JBSS3
sucesso MRFG3
sucesso BEEF3
sucesso MNPR3


In [52]:
# frames['AZUL4']

In [54]:
analisesFundamentalistas = {
                            'Última Cotação ON': 1,
                            'Última Cotação PN': 4,
                            'Preço / Lucro (P/L)': 9,
                            'Retorno sobre PL (ROE)':13,
                            'Valor Patrimonial por Ação (VPA)': 12,
                            'Equity Multiplier (EM)': 14, 
                            'Endividamento Financeiro': 16,
                            'Lucro por Ação (LPA)':25,
                            'Margem EBIT':23,
                            }

anFund = {}

for ii in list(analisesFundamentalistas.keys()):

    lista1 = []
    lista2 = []
    
    ind = analisesFundamentalistas[ii]

    for ie in range(0,len(empresas)):

        val = frames[list(frames.keys())[ie]].iloc[ind,][1]
        if(val.count('.')>1):
            val = val.replace('.','')

        lista1.append(empresas[ie])
        lista2.append(val)

    
    # Preparing DataFrame
    tupleDados = list(zip(lista1,lista2))
    frameDados = pd.DataFrame(tupleDados, columns=['Empresa',ii])
    frameDados = frameDados.set_index('Empresa')

    
    # Final DataFrame
    anFund[ii] = frameDados

In [56]:
dadosParciais = pd.concat(anFund.values(),axis=1)
dadosParciais = dadosParciais.reset_index(level='Empresa')

cols=[i for i in dadosParciais.columns if i not in ["Name","Job"]]

for k in range(1,len(cols)):
    dadosColuna = dadosParciais.iloc[:,k]
    novaColuna = []

    for i in list(dadosColuna):
        if i == 'N/D':
            i = None
            novaColuna.append(i)
        else:
            novaColuna.append(i.replace('%', ''))
        
    dadosParciais.iloc[:,k] = novaColuna


for col in cols:
    dadosParciais[col]=pd.to_numeric(dadosParciais[col], errors='ignore')

dadosParciais

,Empresa,Última Cotação ON,Última Cotação PN,Preço / Lucro (P/L),Retorno sobre PL (ROE),Valor Patrimonial por Ação (VPA),Equity Multiplier (EM),Endividamento Financeiro,Lucro por Ação (LPA),Margem EBIT
0,BRFS3,25.60,NaN,14.34,17.31,10.31,6.14,74.15,1.79,7.10
1,JBSS3,30.12,NaN,6.47,28.78,16.17,3.78,60.45,4.65,7.91
2,MRFG3,18.15,NaN,2.22,127.82,6.38,10.52,88.16,8.16,11.56
3,BEEF3,4.95,NaN,2.72,67.61,2.69,17.95,92.08,1.82,8.97
4,MNPR3,16.00,NaN,6.85,-4.41,-52.92,-0.65,-0.11,2.33,14.58


In [58]:
# Creating Dictionary for the DataFrames
moedasFrames = {}

links = 'https://www.infomoney.com.br/ferramentas/cambio/'

result = requests.get(links)
c = result.content
soup = BeautifulSoup(c,'lxml')

# creating the lists
values = []

for tBody in soup.find('div', {"id": "container_table"}):
    if isinstance(tBody, NavigableString):
        continue
    if isinstance(tBody, Tag):
        for tr in tBody.find_all("tr"):
            for td in tr.find_all("td"):
                if td.string is not None:
                    td = td.string.replace(",",".")
                    values.append(td)

# links = 'http://www.santaritacambios.com.py/'
# result = requests.get(links)
# c = result.content
# soup = BeautifulSoup(c,'lxml')

# for li in soup.find('li', {"class": "cotations-line"}).find_all("div", class_ = ["w-col","w-col-3","w-col-small-3","w-col-tiny-3"]):
#     print(li)
    

# Preparing Dataframe
values = list(filter(None, values)) 

k = 0
new_values = []

while k < len(values):
    new_values.append(values[k:k+4])
    k += 4

moedas = pd.DataFrame(new_values, columns = ["Moeda", "Compra","Venda","Var"])
moedas[['Compra', "Venda", "Var"]] = moedas[['Compra', "Venda", "Var"]].astype("float").round(3)
moedas

,Moeda,Compra,Venda,Var
0,Peso Argentino,0.055,0.055,-1.950
1,Dólar Australiano,4.012,4.017,-1.260
2,Dólar Canadense,4.313,4.315,-0.907
3,Franco Suíço,5.791,5.793,-1.120
4,Dólar Comercial,5.210,5.211,-0.818
5,Dólar Turismo,5.217,5.387,-0.425
6,Euro,6.357,6.359,-1.820
7,Libra Esterlina,7.403,7.404,-0.777
8,Iene,0.048,0.048,-1.030


In [60]:
# Historical Prices

# Dates Analysis
from datetime import date


start = datetime.datetime(2015,1,1)
end = date.today()

# Saving Data

companies = empresas

for i in companies:

    emp = yf.download(i+'.SA',start=start, end=end, progress=False)
    emp.insert(0, 'Empresas',i.replace('.SA',''))
    
    
    if i == companies[0]:
        serieTemp = emp
    else: 
        serieTemp = pd.concat([serieTemp, emp], axis=0)

serieTemp = serieTemp.reset_index(level='Date')


In [62]:
# Importing Tweets

# Biblioteca
import tweepy
from textblob import TextBlob
from googletrans import Translator
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import re
import string

#Tokens de acesso
consumer_key= 'Jnqc4ZEfilay1NaePeKRtlPk8'
consumer_secret= 'xH8TbT48kdbXizzqrMYlyGDPUMdoGyc98Io2ek7X2wIAKC5af8'
access_token='882806009730928640-b0I6qULaf9b1UGkTerbq0q8LYQ7F3lh'
access_token_secret='GNNxnd6jlGO4FBFg5KTawyWSqq6EN8uaQUzdaU9Kuz0Ss'

#Autenticando
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

# Funções

def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+','', text)

    
    return text

In [64]:
for seTwi in empresas:

    # Lista de Comentarios
    listComent = []

    #Buscando tweets
    for tweet in tweepy.Cursor(api.search, q=seTwi, lang='pt').items(25):
    
        # Cleaning Text  
        text = unidecode(tweet.text)
        text = remove_punct(tweet.text)
        text = str(text)
        
        listComent.append(text)
        
    if seTwi == empresas[0]:
        dfTweets = pd.DataFrame(listComent, columns =['Comentarios'])
        dfTweets.insert(0,'Empresas', seTwi)
        
        
    else: 
        newDf = pd.DataFrame(listComent, columns =['Comentarios'])
        newDf.insert(0,'Empresas', seTwi)
        
    dfTweets = pd.concat([dfTweets, newDf], axis=0)

# Dataset Final
dfTweets


,Empresas,Comentarios
0,BRFS3,vaes RiquinhodaFL Nessa caso do BRFS acredito em giro para gerar liquidez e pouca defe...
1,BRFS3,vaes RiquinhodaFL tem várias versões que podemos pensar nesses alugueis to acompanhand...
2,BRFS3,RT MoverTC 💰 As mais lidas da semana Confira a coluna do sergiosanita sobre a Hermes P...
3,BRFS3,Ibovespa Suzano SUZB e BRF BRFS têm maiores quedas httpstcoWuPxwWmSdm
4,BRFS3,Altas e Baixas da Semana Ações do Ibovespa\nLWSA HGTX BIDI SULA YDUQ SUZB BRFS GGBR S...
5,BRFS3,No mensal destaque para BRFS CIEL da massa também e no IBRA destaque pra AMAR POSI ...
6,BRFS3,RT MoverTC 💰 As mais lidas da semana Confira a coluna do sergiosanita sobre a Hermes P...
7,BRFS3,💰 As mais lidas da semana Confira a coluna do sergiosanita sobre a Hermes Pardini a el...
8,BRFS3,Os papéis da Suzano registraram a maior queda da semana de aos R acompanhando em par...
9,BRFS3,As maiores quedas agora entre as ações que compõem o Ibovespa são\n\nAzul AZUL \nIRB...


In [65]:
# Exporting Data
dadosCompletos = pd.ExcelWriter('data.xlsx')

moedas.to_excel(dadosCompletos, sheet_name = 'moedas', index = False)
dadosParciais.to_excel(dadosCompletos, sheet_name = 'acoes', index = False)
serieTemp.to_excel(dadosCompletos, sheet_name = 'serieTemp', index = False)
dfTweets.to_excel(dadosCompletos, sheet_name = 'twiComment', index = False)

dadosCompletos.save()